In [ ]:
import numpy as np
import glob
import re
import shutil
import random
import pandas as pd
import tensorflow as tf
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

In [ ]:
shutil.rmtree("/content/DataBase")
shutil.rmtree("/content/DataBaseOriginal")

In [ ]:
!unzip /content/DataBase.zip

Archive:  /content/DataBase.zip
   creating: DataBase/SETA/
  inflating: DataBase/SETA/healthy_open1.csv  
  inflating: DataBase/SETA/healthy_open10.csv  
  inflating: DataBase/SETA/healthy_open11.csv  
  inflating: DataBase/SETA/healthy_open12.csv  
  inflating: DataBase/SETA/healthy_open2.csv  
  inflating: DataBase/SETA/healthy_open3.csv  
  inflating: DataBase/SETA/healthy_open4.csv  
  inflating: DataBase/SETA/healthy_open5.csv  
  inflating: DataBase/SETA/healthy_open6.csv  
  inflating: DataBase/SETA/healthy_open7.csv  
  inflating: DataBase/SETA/healthy_open8.csv  
  inflating: DataBase/SETA/healthy_open9.csv  
   creating: DataBase/SETB/
  inflating: DataBase/SETB/healthy_closed1.csv  
  inflating: DataBase/SETB/healthy_closed10.csv  
  inflating: DataBase/SETB/healthy_closed11.csv  
  inflating: DataBase/SETB/healthy_closed12.csv  
  inflating: DataBase/SETB/healthy_closed2.csv  
  inflating: DataBase/SETB/healthy_closed3.csv  
  inflating: DataBase/SETB/healthy_closed4.csv  

In [ ]:
files = glob.glob("/content/DataBase/*/*")

In [ ]:
len(files)

48

In [ ]:
def clean(path):
    df = pd.read_csv(path)
    for column in df.columns:
        if df[column].dtype == 'object':
            print("Sample : ",path," feature : ",column," is uncleaned")
            df[column] = pd.to_numeric(df[column], errors='coerce')
            df[column] = df[column].fillna(method='ffill')
            df[column] = df[column].fillna(method='bfill')
    df = df.iloc[:1024,:]
    df.to_csv(path, index=False)

In [ ]:
for i in files:
    clean(i)

Sample :  /content/DataBase/SETB/healthy_closed2.csv  feature :  16  is uncleaned
Sample :  /content/DataBase/SETB/healthy_closed6.csv  feature :  14  is uncleaned
Sample :  /content/DataBase/SETA/healthy_open2.csv  feature :  16  is uncleaned
Sample :  /content/DataBase/SETA/healthy_open11.csv  feature :  14  is uncleaned
Sample :  /content/DataBase/SETD/alzeimer_closed12.csv  feature :  18  is uncleaned
Sample :  /content/DataBase/SETC/alzeimer_open10.csv  feature :  0  is uncleaned
Sample :  /content/DataBase/SETC/alzeimer_open8.csv  feature :  0  is uncleaned
Sample :  /content/DataBase/SETC/alzeimer_open6.csv  feature :  0  is uncleaned
Sample :  /content/DataBase/SETC/alzeimer_open5.csv  feature :  0  is uncleaned


In [ ]:
files = glob.glob("/content/DataBase/*/*")
len(files)

48

In [ ]:
shutil.copytree("/content/DataBase","/content/DataBaseOriginal")

'/content/DataBaseOriginal'

In [ ]:
def positive_shift(path,replace = False,value = 0):
    df = pd.read_csv(path)
    shift = random.randint(3,15)
    df = df.shift(shift)
    if replace:
       df = df.fillna(df.mean())
    df = df.fillna(0)      
    deviation = df.std().tolist()
    noise = np.random.normal([0]*19,deviation,19)
    df += noise/10
    return df

def negative_shift(path,replace = False,value = 0):
    df = pd.read_csv(path)
    shift = random.randint(-15,-3)
    df = df.shift(shift)
    if replace:
       df = df.fillna(df.mean())
    df = df.fillna(0)   
    deviation = df.std().tolist()
    noise = np.random.normal([0]*19,deviation,19)
    df += noise/10
    return df    

def shift(path,label,file_no,target_dir):
    pn = random.randint(0,1)
    if pn == 1:
        file_name = label+str(file_no)+".csv"
        file_name = target_dir+"/"+file_name
        df = positive_shift(path)
        print(path," -> ",file_name," Positive Shift")
        df.to_csv(file_name,index = False)
    else:
        file_name = label+str(file_no)+".csv"
        file_name = target_dir+"/"+file_name
        df = negative_shift(path)
        print(path," -> ",file_name," Negative Shift")
        df.to_csv(file_name,index = False)    


def add_shift_per_sample(path,label):
    file_no = 25
    all_files = glob.glob(path+"/*")
    for i in all_files:
        shift(i,label,file_no,path)
        file_no += 1
   

In [ ]:
add_shift_per_sample("/content/DataBase/SETA",'healthy_open')

/content/DataBase/SETA/healthy_open8.csv  ->  /content/DataBase/SETA/healthy_open25.csv  Positive Shift
/content/DataBase/SETA/healthy_open2.csv  ->  /content/DataBase/SETA/healthy_open26.csv  Negative Shift
/content/DataBase/SETA/healthy_open9.csv  ->  /content/DataBase/SETA/healthy_open27.csv  Positive Shift
/content/DataBase/SETA/healthy_open5.csv  ->  /content/DataBase/SETA/healthy_open28.csv  Positive Shift
/content/DataBase/SETA/healthy_open4.csv  ->  /content/DataBase/SETA/healthy_open29.csv  Positive Shift
/content/DataBase/SETA/healthy_open11.csv  ->  /content/DataBase/SETA/healthy_open30.csv  Negative Shift
/content/DataBase/SETA/healthy_open1.csv  ->  /content/DataBase/SETA/healthy_open31.csv  Positive Shift
/content/DataBase/SETA/healthy_open12.csv  ->  /content/DataBase/SETA/healthy_open32.csv  Positive Shift
/content/DataBase/SETA/healthy_open7.csv  ->  /content/DataBase/SETA/healthy_open33.csv  Positive Shift
/content/DataBase/SETA/healthy_open3.csv  ->  /content/DataBas

In [ ]:
add_shift_per_sample("/content/DataBase/SETB",'healthy_closed')

/content/DataBase/SETB/healthy_closed3.csv  ->  /content/DataBase/SETB/healthy_closed25.csv  Negative Shift
/content/DataBase/SETB/healthy_closed2.csv  ->  /content/DataBase/SETB/healthy_closed26.csv  Negative Shift
/content/DataBase/SETB/healthy_closed5.csv  ->  /content/DataBase/SETB/healthy_closed27.csv  Negative Shift
/content/DataBase/SETB/healthy_closed11.csv  ->  /content/DataBase/SETB/healthy_closed28.csv  Positive Shift
/content/DataBase/SETB/healthy_closed10.csv  ->  /content/DataBase/SETB/healthy_closed29.csv  Positive Shift
/content/DataBase/SETB/healthy_closed6.csv  ->  /content/DataBase/SETB/healthy_closed30.csv  Negative Shift
/content/DataBase/SETB/healthy_closed4.csv  ->  /content/DataBase/SETB/healthy_closed31.csv  Positive Shift
/content/DataBase/SETB/healthy_closed12.csv  ->  /content/DataBase/SETB/healthy_closed32.csv  Positive Shift
/content/DataBase/SETB/healthy_closed9.csv  ->  /content/DataBase/SETB/healthy_closed33.csv  Positive Shift
/content/DataBase/SETB/he

In [ ]:
add_shift_per_sample("/content/DataBase/SETC",'alzeimer_open')

/content/DataBase/SETC/alzeimer_open12.csv  ->  /content/DataBase/SETC/alzeimer_open25.csv  Negative Shift
/content/DataBase/SETC/alzeimer_open11.csv  ->  /content/DataBase/SETC/alzeimer_open26.csv  Negative Shift
/content/DataBase/SETC/alzeimer_open10.csv  ->  /content/DataBase/SETC/alzeimer_open27.csv  Negative Shift
/content/DataBase/SETC/alzeimer_open1.csv  ->  /content/DataBase/SETC/alzeimer_open28.csv  Negative Shift
/content/DataBase/SETC/alzeimer_open8.csv  ->  /content/DataBase/SETC/alzeimer_open29.csv  Positive Shift
/content/DataBase/SETC/alzeimer_open2.csv  ->  /content/DataBase/SETC/alzeimer_open30.csv  Positive Shift
/content/DataBase/SETC/alzeimer_open7.csv  ->  /content/DataBase/SETC/alzeimer_open31.csv  Positive Shift
/content/DataBase/SETC/alzeimer_open9.csv  ->  /content/DataBase/SETC/alzeimer_open32.csv  Positive Shift
/content/DataBase/SETC/alzeimer_open6.csv  ->  /content/DataBase/SETC/alzeimer_open33.csv  Positive Shift
/content/DataBase/SETC/alzeimer_open3.csv  

In [ ]:
add_shift_per_sample("/content/DataBase/SETD",'alzeimer_closed')

/content/DataBase/SETD/alzeimer_closed3.csv  ->  /content/DataBase/SETD/alzeimer_closed25.csv  Negative Shift
/content/DataBase/SETD/alzeimer_closed7.csv  ->  /content/DataBase/SETD/alzeimer_closed26.csv  Positive Shift
/content/DataBase/SETD/alzeimer_closed11.csv  ->  /content/DataBase/SETD/alzeimer_closed27.csv  Positive Shift
/content/DataBase/SETD/alzeimer_closed4.csv  ->  /content/DataBase/SETD/alzeimer_closed28.csv  Positive Shift
/content/DataBase/SETD/alzeimer_closed12.csv  ->  /content/DataBase/SETD/alzeimer_closed29.csv  Negative Shift
/content/DataBase/SETD/alzeimer_closed8.csv  ->  /content/DataBase/SETD/alzeimer_closed30.csv  Positive Shift
/content/DataBase/SETD/alzeimer_closed1.csv  ->  /content/DataBase/SETD/alzeimer_closed31.csv  Negative Shift
/content/DataBase/SETD/alzeimer_closed9.csv  ->  /content/DataBase/SETD/alzeimer_closed32.csv  Positive Shift
/content/DataBase/SETD/alzeimer_closed10.csv  ->  /content/DataBase/SETD/alzeimer_closed33.csv  Negative Shift
/conten

In [ ]:
def mean_roll(path,label,file_no,target_dir):
    df = pd.read_csv(path)
    window = random.randint(3,8)
    df = df.rolling(window = 5,center = True).mean().fillna(0)
    file_name = label+str(file_no)+".csv"
    file_name = target_dir+"/"+file_name
    print(path," -> ",file_name)
    df = df.fillna(method = 'ffill')
    df = df.fillna(0)
    df.to_csv(file_name,index = False)

def add_rolling_per_sample(path,label):
    file_no = 49
    all_files = glob.glob(path+"/*")
    for i in all_files:
        mean_roll(i,label,file_no,path)
        file_no += 1

In [ ]:
add_rolling_per_sample("/content/DataBase/SETA",'healthy_open')

/content/DataBase/SETA/healthy_open30.csv  ->  /content/DataBase/SETA/healthy_open49.csv
/content/DataBase/SETA/healthy_open29.csv  ->  /content/DataBase/SETA/healthy_open50.csv
/content/DataBase/SETA/healthy_open8.csv  ->  /content/DataBase/SETA/healthy_open51.csv
/content/DataBase/SETA/healthy_open2.csv  ->  /content/DataBase/SETA/healthy_open52.csv
/content/DataBase/SETA/healthy_open9.csv  ->  /content/DataBase/SETA/healthy_open53.csv
/content/DataBase/SETA/healthy_open5.csv  ->  /content/DataBase/SETA/healthy_open54.csv
/content/DataBase/SETA/healthy_open4.csv  ->  /content/DataBase/SETA/healthy_open55.csv
/content/DataBase/SETA/healthy_open11.csv  ->  /content/DataBase/SETA/healthy_open56.csv
/content/DataBase/SETA/healthy_open34.csv  ->  /content/DataBase/SETA/healthy_open57.csv
/content/DataBase/SETA/healthy_open1.csv  ->  /content/DataBase/SETA/healthy_open58.csv
/content/DataBase/SETA/healthy_open28.csv  ->  /content/DataBase/SETA/healthy_open59.csv
/content/DataBase/SETA/heal

In [ ]:
add_rolling_per_sample("/content/DataBase/SETB",'healthy_closed')

/content/DataBase/SETB/healthy_closed27.csv  ->  /content/DataBase/SETB/healthy_closed49.csv
/content/DataBase/SETB/healthy_closed34.csv  ->  /content/DataBase/SETB/healthy_closed50.csv
/content/DataBase/SETB/healthy_closed30.csv  ->  /content/DataBase/SETB/healthy_closed51.csv
/content/DataBase/SETB/healthy_closed3.csv  ->  /content/DataBase/SETB/healthy_closed52.csv
/content/DataBase/SETB/healthy_closed25.csv  ->  /content/DataBase/SETB/healthy_closed53.csv
/content/DataBase/SETB/healthy_closed2.csv  ->  /content/DataBase/SETB/healthy_closed54.csv
/content/DataBase/SETB/healthy_closed32.csv  ->  /content/DataBase/SETB/healthy_closed55.csv
/content/DataBase/SETB/healthy_closed26.csv  ->  /content/DataBase/SETB/healthy_closed56.csv
/content/DataBase/SETB/healthy_closed5.csv  ->  /content/DataBase/SETB/healthy_closed57.csv
/content/DataBase/SETB/healthy_closed11.csv  ->  /content/DataBase/SETB/healthy_closed58.csv
/content/DataBase/SETB/healthy_closed10.csv  ->  /content/DataBase/SETB/h

In [ ]:
add_rolling_per_sample("/content/DataBase/SETC",'alzeimer_open')

/content/DataBase/SETC/alzeimer_open26.csv  ->  /content/DataBase/SETC/alzeimer_open49.csv
/content/DataBase/SETC/alzeimer_open34.csv  ->  /content/DataBase/SETC/alzeimer_open50.csv
/content/DataBase/SETC/alzeimer_open12.csv  ->  /content/DataBase/SETC/alzeimer_open51.csv
/content/DataBase/SETC/alzeimer_open11.csv  ->  /content/DataBase/SETC/alzeimer_open52.csv
/content/DataBase/SETC/alzeimer_open35.csv  ->  /content/DataBase/SETC/alzeimer_open53.csv
/content/DataBase/SETC/alzeimer_open31.csv  ->  /content/DataBase/SETC/alzeimer_open54.csv
/content/DataBase/SETC/alzeimer_open33.csv  ->  /content/DataBase/SETC/alzeimer_open55.csv
/content/DataBase/SETC/alzeimer_open28.csv  ->  /content/DataBase/SETC/alzeimer_open56.csv
/content/DataBase/SETC/alzeimer_open32.csv  ->  /content/DataBase/SETC/alzeimer_open57.csv
/content/DataBase/SETC/alzeimer_open10.csv  ->  /content/DataBase/SETC/alzeimer_open58.csv
/content/DataBase/SETC/alzeimer_open1.csv  ->  /content/DataBase/SETC/alzeimer_open59.csv


In [ ]:
add_rolling_per_sample("/content/DataBase/SETD",'alzeimer_closed')

/content/DataBase/SETD/alzeimer_closed3.csv  ->  /content/DataBase/SETD/alzeimer_closed49.csv
/content/DataBase/SETD/alzeimer_closed28.csv  ->  /content/DataBase/SETD/alzeimer_closed50.csv
/content/DataBase/SETD/alzeimer_closed7.csv  ->  /content/DataBase/SETD/alzeimer_closed51.csv
/content/DataBase/SETD/alzeimer_closed29.csv  ->  /content/DataBase/SETD/alzeimer_closed52.csv
/content/DataBase/SETD/alzeimer_closed25.csv  ->  /content/DataBase/SETD/alzeimer_closed53.csv
/content/DataBase/SETD/alzeimer_closed36.csv  ->  /content/DataBase/SETD/alzeimer_closed54.csv
/content/DataBase/SETD/alzeimer_closed11.csv  ->  /content/DataBase/SETD/alzeimer_closed55.csv
/content/DataBase/SETD/alzeimer_closed4.csv  ->  /content/DataBase/SETD/alzeimer_closed56.csv
/content/DataBase/SETD/alzeimer_closed12.csv  ->  /content/DataBase/SETD/alzeimer_closed57.csv
/content/DataBase/SETD/alzeimer_closed35.csv  ->  /content/DataBase/SETD/alzeimer_closed58.csv
/content/DataBase/SETD/alzeimer_closed8.csv  ->  /con

In [ ]:
shutil.make_archive('Augmented', 'zip','/content/DataBase')

'/content/Augmented.zip'

In [ ]:
def return_splits():
    augmented_files = glob.glob("/content/DataBase/*/*")
    train_files = list(set(augmented_files) - set(files))
    test_files = files
    for i in range(0,len(test_files),12):
        train_files += test_files[i:i+4]
    test_files = list(set(test_files)-set(train_files))    
    return train_files,test_files 
train,test = return_splits()     

In [ ]:
len(train),len(test)

(160, 32)

In [ ]:
def decompose(wave_data):
    pca = PCA(n_components=5)
    pca.fit(wave_data.T)
    pc = pca.components_
    wave_data_pca = np.dot(pc, wave_data)
    wave_data_pca = wave_data_pca.mean(axis = 0)
    return wave_data_pca.tolist()

In [ ]:
def handle_nan(df_f):
    df = pd.read_csv(df_f)
    count = 0
    if df.isnull().sum().sum() != 0:
        print(df_f,end = " ")
        while df.isnull().sum().sum() != 0:
            count = count + 1
            df = df.fillna(method = 'ffill')
            df = df.fillna(method = 'bfill')
        print("   " , count, "null values found and taken care of","\n")    
    return df

In [ ]:
def create_ds(all_files):
    dataset = []
    for file in all_files:
        df = handle_nan(file)
        scaler = StandardScaler()
        df = scaler.fit_transform(df)
        sample = decompose(df)

        if "closed" in file:
            sample.append("closed")
        elif "open" in file:
            sample.append("open")

        if "healthy" in file:
            sample.append("healthy")
        elif "alzeimer" in file:
            sample.append("alzeimer")

        dataset.append(sample)
    return pd.DataFrame(dataset)        

In [ ]:
train_df = create_ds(train)

/content/DataBase/SETC/alzeimer_open10.csv     1 null values found and taken care of 



In [ ]:
train_df.to_csv("train.csv",index= False)

In [ ]:
test_df = create_ds(test)

/content/DataBase/SETC/alzeimer_open6.csv     1 null values found and taken care of 

/content/DataBase/SETC/alzeimer_open8.csv     1 null values found and taken care of 

/content/DataBase/SETC/alzeimer_open5.csv     1 null values found and taken care of 



In [ ]:
test_df.to_csv("test.csv",index= False)